In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from block_differential_evolution import block_differential_evolution

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

Running on device: CUDA


In [2]:
# Define a simple neural network class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [3]:
# Set a random seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# Define the transformation to apply to the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the original MNIST dataset
mnist_train = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST('data', train=False, download=True, transform=transform)

# Create an empty list to store the balanced dataset
balanced_train_dataset = []
balanced_test_dataset = []

# Set the number of samples per class
samples_per_class = 100

# Randomly select samples from each class for the training dataset
for i in range(10):
    class_indices = np.where(np.array(mnist_train.targets) == i)[0]
    selected_indices = np.random.choice(class_indices, samples_per_class, replace=False)
    balanced_train_dataset.extend(Subset(mnist_train, selected_indices))

# Randomly select samples from each class for the test dataset
for i in range(10):
    class_indices = np.where(np.array(mnist_test.targets) == i)[0]
    selected_indices = np.random.choice(class_indices, samples_per_class, replace=False)
    balanced_test_dataset.extend(Subset(mnist_test, selected_indices))

# Create DataLoaders for the balanced datasets
batch_size = 64
train_loader = DataLoader(balanced_train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(balanced_test_dataset, batch_size=batch_size, shuffle=False)

# Check the sizes of the resulting datasets
print("Size of balanced training dataset:", len(balanced_train_dataset))

# Now you can use train_loader and test_loader for training and testing your model.

Size of balanced training dataset: 1000


In [4]:
# Load and preprocess the MNIST dataset
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

#train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Size of balanced test dataset:", len(test_dataset))

Size of balanced test dataset: 10000


In [5]:
# Step 4: Choose a loss function and optimizer
model = NeuralNetwork().to(DEVICE)
criterion = nn.CrossEntropyLoss()

In [6]:
params_sizes={}
for param_tensor in model.state_dict():
    params_sizes[param_tensor] = model.state_dict()[param_tensor].size()
    # print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [7]:
def set_weights(model_state, all_parameters):
    counted_params = 0
    for param_tensor in model_state:
        # print(param_tensor, counted_params, params_sizes[param_tensor].numel())
        if not 'num_batches_tracked' in param_tensor:
            model_state[param_tensor] = torch.tensor(all_parameters[counted_params:params_sizes[param_tensor].numel()+counted_params]).reshape(params_sizes[param_tensor])
            counted_params += params_sizes[param_tensor].numel()
    return model_state

In [8]:
def f1score_in_optimization(model, loader):
    # running_loss = 0.0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            # loss = criterion(output, target)

            # running_loss += loss.item()

            _, predicted = torch.max(output.data, 1)

            true_labels.extend(target.tolist())
            predicted_labels.extend(predicted.tolist())
        
    train_f1score = f1_score(true_labels, predicted_labels, average='weighted')
    return train_f1score

In [9]:
def fitness_func(parameters):
    #print(parameters[:20])
    model.load_state_dict(set_weights(model.state_dict(), parameters))
    model.to(DEVICE)
    model.eval();
    fitness = -1 * f1score_in_optimization(model, train_loader)
    # print(fitness)
    return fitness

In [10]:
def get_parameters(model):
    model_state = model.state_dict()
    params=[]
    for p in model_state:
        if not 'num_batches_tracked' in p:
            params.append(model_state[p].view(-1))

    params = torch.cat(params).cpu().detach().numpy()
    return params

In [11]:
params = get_parameters(model)

In [12]:
def population_initializer(Network, popsize):
    model = Network().to(DEVICE)
    params = get_parameters(model)
    initial_population = np.array([params])
    for i in range(popsize-1):
        model = Network().to(DEVICE)
        #print(f1score_in_optimization(model, train_loader))
        params = get_parameters(model)
        #print(fitness_func(params))
        initial_population = np.concatenate([initial_population, [params]], axis=0)
    return initial_population

In [18]:
block_size = 10
max_iterations = 1000000
popsize = 100

initial_population = population_initializer(NeuralNetwork, popsize)
dimensions = initial_population.shape[1]
print(initial_population.shape)

(100, 109386)


In [19]:
initial_population.max(axis=0).shape

(109386,)

In [20]:
#for i in range(popsize):
print(fitness_func(initial_population[0]))
    

-0.026253856075750228


In [21]:
bounds = np.concatenate([initial_population.min(axis=0).reshape(-1, 1), initial_population.max(axis=0).reshape(-1,1)], axis=1)

In [22]:
result = block_differential_evolution(fitness_func, bounds, maxiter=max_iterations, block_size=block_size, save_link=f'ann_bde_b{block_size}_FE{max_iterations*100}_mnist_training_history.npz', popsize=popsize, callback=None, polish=False, disp=True, x0=params, updating='deferred', init=initial_population)

best f:  -0.06489885447083368
best f:  -0.06989821006748599
differential_evolution step 1: f(x)= -0.0698982
best f:  -0.07315875613747955
differential_evolution step 2: f(x)= -0.0731588
best f:  -0.07315875613747955
differential_evolution step 3: f(x)= -0.0731588
best f:  -0.07315875613747955
differential_evolution step 4: f(x)= -0.0731588
best f:  -0.07315875613747955
differential_evolution step 5: f(x)= -0.0731588
best f:  -0.07315875613747955
differential_evolution step 6: f(x)= -0.0731588
best f:  -0.0731640160150298
differential_evolution step 7: f(x)= -0.073164
best f:  -0.0731640160150298
differential_evolution step 8: f(x)= -0.073164
best f:  -0.0731640160150298
differential_evolution step 9: f(x)= -0.073164
best f:  -0.07893410219167986
differential_evolution step 10: f(x)= -0.0789341
best f:  -0.07893410219167986
differential_evolution step 11: f(x)= -0.0789341
best f:  -0.07893410219167986
differential_evolution step 12: f(x)= -0.0789341
best f:  -0.07893410219167986
differe

best f:  -0.1569977590360512
differential_evolution step 106: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 107: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 108: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 109: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 110: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 111: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 112: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 113: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 114: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 115: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 116: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 117: f(x)= -0.156998
best f:  -0.1569977590360512
differential_evolution step 118: f(

best f:  -0.25320261145004763
differential_evolution step 211: f(x)= -0.253203
best f:  -0.25320261145004763
differential_evolution step 212: f(x)= -0.253203
best f:  -0.25320261145004763
differential_evolution step 213: f(x)= -0.253203
best f:  -0.25320261145004763
differential_evolution step 214: f(x)= -0.253203
best f:  -0.2549930676561383
differential_evolution step 215: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 216: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 217: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 218: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 219: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 220: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 221: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 222: f(x)= -0.254993
best f:  -0.2549930676561383
differential_evolution step 223

best f:  -0.2937882425918838
differential_evolution step 316: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 317: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 318: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 319: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 320: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 321: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 322: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 323: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 324: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 325: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 326: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 327: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 328: f(

best f:  -0.2937882425918838
differential_evolution step 422: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 423: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 424: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 425: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 426: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 427: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 428: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 429: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 430: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 431: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 432: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 433: f(x)= -0.293788
best f:  -0.2937882425918838
differential_evolution step 434: f(

...

best f:  -0.3899359878760324
differential_evolution step 12029: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12030: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12031: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12032: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12033: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12034: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12035: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12036: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12037: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12038: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12039: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12040: f(x)= -0.389936
best f:  -0.3899359878760324
differentia

best f:  -0.3899359878760324
differential_evolution step 12132: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12133: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12134: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12135: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12136: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12137: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12138: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12139: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12140: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12141: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12142: f(x)= -0.389936
best f:  -0.3899359878760324
differential_evolution step 12143: f(x)= -0.389936
best f:  -0.3899359878760324
differentia

best f:  -0.39030463103268137
differential_evolution step 12235: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12236: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12237: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12238: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12239: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12240: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12241: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12242: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12243: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12244: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12245: f(x)= -0.390305
best f:  -0.39030463103268137
differential_evolution step 12246: f(x)= -0.390305
best f:  -0.3903046310326813

best f:  -0.3903412725737198
differential_evolution step 12337: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12338: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12339: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12340: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12341: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12342: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12343: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12344: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12345: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12346: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12347: f(x)= -0.390341
best f:  -0.3903412725737198
differential_evolution step 12348: f(x)= -0.390341
best f:  -0.3903412725737198
differentia

KeyboardInterrupt: 

In [ ]:
print(f'Train best fitness F1-score: {result.fun * -100:.2f}%)

In [ ]:
model.load_state_dict(set_weights(model.state_dict(), result.x))

In [ ]:
# Save the trained model and metrics
torch.save(model.state_dict(), f'ann_bde_b{block_size}_mnist_model.pth')

In [ ]:
# Load the trained model
block_size=10
model.load_state_dict(torch.load(f'ann_bde_b{block_size}_mnist_model.pth'))

In [ ]:
# Step 6: Test the network
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        true_labels.extend(target.tolist())
        predicted_labels.extend(predicted.tolist())
        
    test_accuracy = accuracy_score(true_labels, predicted_labels)
    test_f1score = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Test Accuracy: {test_accuracy*100:.2f}%, Test F1-score: {test_f1score*100:.2f}%')

In [ ]:
# Plot the training loss and accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Trained ANN by Block DE optimizer')
plt.subplot(1, 2, 2)
plt.plot(train_f1score_history, label='Training F1-score')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()